# Trabajo N°2 Web scraping - spaCy

El siguiente trabajo consiste en utilizar la libreria de spaCy para procesar un texto extraido de la web con web scraping, con la finalidad de realizar:



*   Tokenizacion
*   Reconocimiento de entidades
*   Análisis de dependencias sintactica
*   Graficar una nube de palabras




## 1 - Web Scraping

Utilizamos Beautiful Soup para extraer un fragmento del cuento "El gato con botas" que se encuentra en la pagina.

In [ ]:
# Importamos Librerias
import requests
from bs4 import BeautifulSoup

In [ ]:
# Guardamos en una variable la url
url = "https://www.cuentosinfantiles.net/cuentos-el-gato-con-botas/"

In [ ]:
# Guardamos el contenido de la url en una variable
contenido = requests.get(url).text

In [ ]:
# Parseamos el contenio con BeautifulSoup
soup = BeautifulSoup(contenido, "html.parser")

In [ ]:
# Buscamos la etiqueta correspondiente al titulo y lo mostramos
titulo = soup.find("h1").get_text()
print(f"Titulo del cuento: '{titulo}'\n")

# Creamos una variable con todos los parrafos que se encuentra en la pagina y lo mostramos
texto = "\n".join([p.get_text() for p in soup.find_all("p")] )
print(texto)

In [ ]:
# Recortamos una parte del texto
palabra_objetivo = "ahogando!”-"
palabras = texto.split()
texto_extraido = " ".join(palabras[:palabras.index(palabra_objetivo) + 1])
print(texto_extraido)

In [ ]:
# Limpiamos el texto extraido de simbolos
texto_pequeño = texto_extraido.replace("”", "").replace("-", "").replace("“", "").replace("!", " ").replace("¡", " ")
print(texto_pequeño)

## 2 - Análisis de dependencia sintactica con spaCy

La dependencia sintáctica es una relación entre dos elementos lingüísticos en la cual uno de ellos está subordinado, regido o exigido por el otro.

In [ ]:
# Descargamos un modelo de lenguaje pre-entrenado en español de la biblioteca spaCy
!python -m spacy download es_core_news_lg -q

In [ ]:
# Importamos Librerias
import spacy
import es_core_news_lg
from spacy import displacy

In [ ]:
# Pipeline de Spacy: Cargamos el modelo
nlp = spacy.load("es_core_news_lg")
nlp = es_core_news_lg.load()

In [ ]:
# Procesar el texto con Spacy
texto_recortado = nlp(texto_pequeño)

In [ ]:
# Tokenizacion
tokens = [token.text for token in texto_recortado]
print(tokens)

In [ ]:
# Lematizacion: forma base de cada token
for token in texto_recortado:
    if not token.is_punct and not token.is_space: # Ignoramos puntuación y espacios
        print(f"'{token.text}' ===> '{token.lemma_}'")

In [ ]:
# Realizamos Etiquetado gramatical
for token in texto_recortado:
    if not token.is_space: # Ignoramos espacios
        print(f"'{token.text}' ===> {token.pos_} ({spacy.explain(token.pos_)}) ===> {token.tag_}")

In [ ]:
# Analisis de dependencia sintactica
for token in texto_recortado:
     if not token.is_space:
        print(f"'{token.text}' ==> {token.dep_} ({spacy.explain(token.dep_)}) ==> '{token.head.text}'")

In [ ]:
# Visualización de dependencias y entidades
displacy.render(texto_recortado, style='dep', jupyter=True, options={'distance': 100, 'color': '#60C878', 'bg': 'black', 'font': 'Courier New'})

In [ ]:
# Buscamos y mostramos las entidades en el fragmento del cuento
if texto_recortado.ents:
    print("Entidades encontradas en el fragmento del cuento:")
    print("[Texto de la Entidad] -> [Etiqueta]")
    for ent in texto_recortado.ents:
        print(f"[{ent.text}] ==> {ent.label_} ({spacy.explain(ent.label_)})")
else:
    print("No se encontraron entidades.")

In [ ]:
# Resaltamos la entidad en el texto
displacy.render(texto_recortado,style='ent',jupyter=True,options={'distance':200})

##3 - Calculamos la frecuencia de palabras clave

Necesarias para crear la nube de palabras

In [ ]:
# Importamos Libreria
from collections import Counter

In [ ]:
# Creamos una lista vacia
palabras_clave = []

In [ ]:
# Normalización: Obtener lema y convertir a minúsculas
for token in texto_recortado:
    if token.is_alpha and not token.is_stop:
      palabras_clave.append(token.lemma_.lower()) # Agregamos palabras clave en minuscula a la lista vacia

print(f"Se extrajeron {len(palabras_clave)} palabras clave.")
# Ejemplo de las primeras palabras extraídas:
print(f"Ejemplo: {palabras_clave[:20]}")

In [ ]:
# Contamos las veces que se repite la palabra clave y lo almacenamos en una variable
frecuencia_palabras = Counter(palabras_clave)

In [ ]:
# Establecemos un numero de palabras mas comunes y mostramos la frecuencia de la misma
N = 15
palabras_mas_comunes = frecuencia_palabras.most_common(N)
for palabra, frecuencia in palabras_mas_comunes:
    print(f"[*] '{palabra}' : {frecuencia}")

## 4 - Graficamos una Nube de Palabras

In [ ]:
# Importamos Librerias
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# Creamos y configuramos la nube de palabras
wordcloud_generator = WordCloud(
    width=700, # Ancho
    height=350, # Alto
    background_color='green', # Color de fondo de pantalla
    colormap='inferno', # Paleta de colores
    max_words=30,      # Mostrar máximo 50 palabras
    stopwords=None,    # Colocamos None por lo que filtramos stop words antes
    collocations=False # Evitar que agrupe palabras
).generate_from_frequencies(frecuencia_palabras) # Generamos la nube con las frecuencias calculadas

In [ ]:
# Configuramos el tamaño del ploteo
plt.figure(figsize=(8, 6)) # Tamaño de la figura
plt.imshow(wordcloud_generator, interpolation='bilinear') # Muestra la imagen con filtro
plt.axis("off") # Para no mostrar los ejes X e Y
plt.tight_layout(pad=0) # Ajustar para que no haya bordes extra
plt.show() # Muestra el grafico creado

Conformando una representación visual de las palabras más frecuentes del fragmento del cuento.